In [2]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model

In [3]:
data = pd.read_csv('datasets/IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [4]:
english_stops = set(stopwords.words('english'))

In [5]:
def load_dataset():
    df = pd.read_csv('datasets/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


C:\Users\divya\AppData\Local\Temp\ipykernel_6308\1190148056.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
17770    [hi, may, i, theater, major, sophisticated, mo...
5199     [pendragon, pictures, new, film, h, g, wells, ...
17341    [this, would, much, fun, see, theater, back, t...
18889    [i, watched, episode, lot, times, i, get, prue...
427      [this, ok, adaptation, breath, taking, book, d...
                               ...                        
33455    [this, stupidest, movie, i, ever, seen, life, ...
14803    [geez, gay, man, lives, nyc, i, gratefully, sa...
14708    [some, greatest, loved, horror, movies, wicked...
19849    [imagine, plight, richard, painter, whose, rea...
37660    [this, this, one, this, worst, movie, ever, ma...
Name: review, Length: 40000, dtype: object 

31487    [take, famous, play, director, ideas, using, c...
18979    [wow, that, james, purefoy, looks, exactly, li...
28895    [saltmen, long, film, genre, quite, often, pac...
37818    [this, first, time, i, feel, need, comment, im...
13188    [this, movie, changes, way, third, way, totall...
 

In [7]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [8]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 4595   108     1 ...     0     0     0]
 [35706  1222    82 ...   219   747  3649]
 [    8    12    17 ...     6   717    20]
 ...
 [  386   708   342 ...    35   151   287]
 [  719  4883   713 ... 13840 11736   283]
 [    8     8     5 ...    42    18  1179]] 

Encoded X Test
 [[   98   709   197 ...     0     0     0]
 [ 1239   146   492 ...     0     0     0]
 [39266   104     4 ...     0     0     0]
 ...
 [ 1263  1213  1503 ...    47  3847 10684]
 [    8     4   224 ...   301  2105     4]
 [  143  1300  2135 ...     0     0     0]] 

Maximum review length:  130


In [9]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

D:\Programming\python\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [11]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 0.5271 - loss: 0.6884  
Epoch 1: accuracy improved from -inf to 0.54577, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 74s 213ms/step - accuracy: 0.5272 - loss: 0.6884
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step - accuracy: 0.5842 - loss: 0.6616  
Epoch 2: accuracy improved from 0.54577 to 0.58780, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 72s 231ms/step - accuracy: 0.5842 - loss: 0.6616
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.6612 - loss: 0.6069  
Epoch 3: accuracy improved from 0.58780 to 0.70200, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 264ms/step - accuracy: 0.6614 - loss: 0.6068
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.7088 - loss: 0.5588  
Epoch 4: accuracy improved from 0.70200 to 0.70353, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 266ms/step - accuracy: 0.7088 - loss: 0.5589
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 0.7362 - loss: 0.5486  
Epoch 5: accuracy improved from 0.70353 to 0.75650, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 143s 268ms/step - accuracy: 0.7362 - loss: 0.5485


In [12]:
import numpy as np

# Get prediction probabilities
y_pred_prob = model.predict(x_test, batch_size=128)

# Convert probabilities to class labels (0 or 1)
y_pred = (y_pred_prob > 0.5).astype("int32")

true = np.sum(y_pred.reshape(-1) == y_test)

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {:.2f}%'.format(true / len(y_pred) * 100))


79/79 ━━━━━━━━━━━━━━━━━━━━ 10s 116ms/step
Correct Prediction: 8114
Wrong Prediction: 1886
Accuracy: 81.14%


In [13]:
loaded_model = load_model('models/LSTM.h5')

In [14]:
review = str(input('Movie Review: '))

Movie Review:  Movie was very nice and good


In [16]:
# Pre-process input
import re

regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  Movie was very nice and good
Filtered:  ['movie nice good']


In [17]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  3 238   9   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]]


In [18]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step
[[0.8319793]]


In [19]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
